In [1]:
import sqlite3

current_date = "2025-10-23"
database_path = f"fundamental_analysis/{current_date}/fundamental_analysis.db"

with open(database_path, "w") as f:
    f.write("")

command = """CREATE TABLE IF NOT EXISTS fundamental_analysis (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
	Papel VARCHAR(10) NOT NULL, 
	Cotacao FLOAT NOT NULL, 
	Tipo VARCHAR(10) NOT NULL, 
	DataUltimaCotacao DATETIME NOT NULL,
	Empresa VARCHAR(100) NOT NULL,
	Min52Sem FLOAT NOT NULL,
	Setor VARCHAR(100) NOT NULL,
	Max52Sem FLOAT NOT NULL,
	Subsetor VARCHAR(100) NOT NULL,
	VolumeMedio2M FLOAT NOT NULL,
	ValorMercado FLOAT NOT NULL,
	UltimoBalancoProcessado DATETIME NOT NULL,
	ValorFirma FLOAT NOT NULL,
	NumeroAcoes INTEGER NOT NULL,
	CrescimentoReceita5a FLOAT NOT NULL,
	Ativo FLOAT NOT NULL,
	DividaBruta FLOAT NOT NULL,
	Disponibilidades FLOAT NOT NULL,
	DividaLiquida FLOAT NOT NULL,
	AtivoCirculante FLOAT NOT NULL,
	PatrimonioLiquido FLOAT NOT NULL,
	ReceitaLiquida FLOAT NOT NULL,
	ReceitaLiquida3Meses FLOAT NOT NULL,
	EBIT FLOAT NOT NULL,
	EBIT3Meses FLOAT NOT NULL,
	LucroLiquido FLOAT NOT NULL,
	LucroLiquido3Meses FLOAT NOT NULL,
	DivYield FLOAT NOT NULL,
	Depositos FLOAT NOT NULL,
	CartaoCredito FLOAT NOT NULL,
	ResultadoIntFinanceiro FLOAT NOT NULL,
	ResultadoIntFinanceiro3Meses FLOAT NOT NULL,
	ReceitaServicos FLOAT NOT NULL,
	ReceitaServicos3Meses FLOAT NOT NULL
)"""

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
cursor.executescript(command)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

In [2]:
import pandas as pd
import json

df_paths = [
    "fundamental_analysis/2025-04-17/fundamental_analysis.csv",
    "fundamental_analysis/2025-07-18/fundamental_analysis.csv",
    "fundamental_analysis/2025-08-31/fundamental_analysis.csv",
    "fundamental_analysis/2025-10-23/fundamental_analysis.csv",
]

dfs = []
for path in df_paths:
    df = pd.read_csv(path)
    df = df.fillna(0)
    dfs.append(df)

df = pd.concat(dfs)
df = df.drop_duplicates(subset=["Papel", "Últ balanço processado"], keep="first")

In [3]:
import pandas as pd

with open("fundamental_analysis_columns.json") as f:
    column_maps = json.load(f)

command = []
for idx, row in df.iterrows():
    keys, values = [], []
    for column in column_maps:
        keys.append(column_maps[column])
        if column_maps[column] in ["DataUltimaCotacao", "UltimoBalancoProcessado"]:
            # convert from dd/mm/yyyy to yyyy-mm-dd
            try:
                day, month, year = row[column].split("/")
                values.append(f"'{year}-{month}-{day}'")
            except Exception:
                values.append(f"'{row[column]}'")
        else:
            values.append(f"'{row[column]}'")

    query = f"INSERT INTO fundamental_analysis ({', '.join(keys)}) VALUES ({', '.join(values)});"
    command.append(query)

# remove duplicates
command = set(command)
command = "\n".join(command)

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
cursor.executescript(command)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

In [6]:
command = """SELECT Papel, Empresa, DivYield, UltimoBalancoProcessado 
FROM fundamental_analysis 
WHERE DivYield > 9 AND UltimoBalancoProcessado > '2025-06-01'"""

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(database_path)

# Create a cursor object using the connection
cursor = connection.cursor()

# Query the database
cursor.execute(command)

# Fetch column names
columns = (
    [description[0] for description in cursor.description] if cursor.description else []
)

# Fetch results
rows = cursor.fetchall()

# Close the cursor and connection
cursor.close()
connection.close()

# Format results as markdown table
if not rows:
    output = "No data found with given query"
else:
    # Create markdown table
    output = "| " + " | ".join(columns) + " |\n"
    output += "| " + " | ".join(["---" for _ in columns]) + " |\n"
    for row in rows:
        output += (
            "| "
            + " | ".join([str(cell) if cell is not None else "" for cell in row])
            + " |\n"
        )

print(output)

| Papel | Empresa | DivYield | UltimoBalancoProcessado |
| --- | --- | --- | --- |
| ISAE4 | ISA ENERGIA PN | 10.2 | 2025-06-30 |
| PETR3 | PETROBRAS ON | 15.4 | 2025-06-30 |
| WHRL4 | WHIRLPOOL SA PN | 17.1 | 2025-06-30 |
| LOGG3 | LOG COM PROP ON NM | 10.4 | 2025-06-30 |
| BBSE3 | BBSEGURIDADE ON NM | 13.0 | 2025-06-30 |
| SOND5 | SONDOTECNICA S/A PNA | 11.2 | 2025-06-30 |
| MTRE3 | MITRE REALTY ON NM | 14.0 | 2025-06-30 |
| EKTR3 | ELEKTRO ELETRICIDADE E SERVIÇOS SA ON | 10.5 | 2025-06-30 |
| ROMI3 | ROMI ON | 9.4 | 2025-09-30 |
| WHRL3 | WHIRLPOOL SA ON | 16.7 | 2025-06-30 |
| RANI3 | IRANI ON | 9.1 | 2025-06-30 |
| UNIP5 | UNIPAR PARTICIPAÇÕES SA PNA | 14.1 | 2025-06-30 |
| CAMB3 | PENALTY ON | 9.7 | 2025-06-30 |
| CGRA4 | GRAZZIOTIN PN | 11.7 | 2025-06-30 |
| CASH3 | MELIUZ ON NM | 41.3 | 2025-06-30 |
| HBTS5 | CIA HABITASUL PNA | 18.4 | 2025-06-30 |
| ROMI3 | ROMI ON | 9.8 | 2025-06-30 |
| GEPA3 | DUKE ENERGY INT.GERAÇÃO PARANAPANEMA SA ON | 20.6 | 2025-06-30 |
| GEPA4 | DUKE EN

In [7]:
try:
    with sqlite3.connect(database_path) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT sql
            FROM sqlite_master
            WHERE type='table' AND sql IS NOT NULL
        """
        )
        schemas = cursor.fetchall()
        schema = "\n".join(schema[0] for schema in schemas if schema[0])
except Exception as e:
    raise Exception(f"Failed to get table schema: {e}")

print(schema)

CREATE TABLE fundamental_analysis (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
	Papel VARCHAR(10) NOT NULL, 
	Cotacao FLOAT NOT NULL, 
	Tipo VARCHAR(10) NOT NULL, 
	DataUltimaCotacao DATETIME NOT NULL,
	Empresa VARCHAR(100) NOT NULL,
	Min52Sem FLOAT NOT NULL,
	Setor VARCHAR(100) NOT NULL,
	Max52Sem FLOAT NOT NULL,
	Subsetor VARCHAR(100) NOT NULL,
	VolumeMedio2M FLOAT NOT NULL,
	ValorMercado FLOAT NOT NULL,
	UltimoBalancoProcessado DATETIME NOT NULL,
	ValorFirma FLOAT NOT NULL,
	NumeroAcoes INTEGER NOT NULL,
	CrescimentoReceita5a FLOAT NOT NULL,
	Ativo FLOAT NOT NULL,
	DividaBruta FLOAT NOT NULL,
	Disponibilidades FLOAT NOT NULL,
	DividaLiquida FLOAT NOT NULL,
	AtivoCirculante FLOAT NOT NULL,
	PatrimonioLiquido FLOAT NOT NULL,
	ReceitaLiquida FLOAT NOT NULL,
	ReceitaLiquida3Meses FLOAT NOT NULL,
	EBIT FLOAT NOT NULL,
	EBIT3Meses FLOAT NOT NULL,
	LucroLiquido FLOAT NOT NULL,
	LucroLiquido3Meses FLOAT NOT NULL,
	DivYield FLOAT NOT NULL,
	Depositos FLOAT NOT NULL,
	CartaoCredito FLOAT NOT N